In [9]:
import os
from pydub import AudioSegment
import librosa
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from dsp import *

我们希望使用 SVM 实现语音识别，步骤如下：

1. 特征提取

2. 模型训练

3. 模型识别

In [10]:
def readDat(filepath):
    # 使用16位有符号整数(int16)格式读取原始数据
    y = np.fromfile(filepath, dtype=np.int16)
    # 将整数值归一化到[-1, 1]范围
    y = y.astype(np.float32) / 32768.0
    return y

In [ ]:
# 特征提取
def extractFeature(audio_file, sr=8000, win=256, step=160):
    y = readDat(audio_file)
    
    segments = vad(y, sr, win, step)

    if len(segments) == 0:
        print(f"警告: {audio_file} 没有检测到语音段")
        return None
    
    # 取所有音频段的起始和结束时间
    start = segments[0][0]
    end = segments[-1][1]
    y_speech = y[int(start):int(end)]
    
    # 确保语音段足够长
    min_length = win * 2  # 确保能生成至少一个完整的STFT帧
    if len(y_speech) < min_length:
        print(f"警告: {audio_file} 语音段太短({len(y_speech)}采样点)，填充至{min_length}采样点")
        y_speech = np.pad(y_speech, (0, min_length - len(y_speech)), 'constant')
        # return None


    # 提取MFCC特征
    mfcc_feature = computeMFCC(y_speech, sr, win, D=13, M=26)

    mfcc_mean = np.mean(mfcc_feature, axis=1)
    mfcc_std = np.std(mfcc_feature, axis=1)
    mfcc_max = np.max(mfcc_feature, axis=1)
    mfcc_min = np.min(mfcc_feature, axis=1)


    feature = np.concatenate([mfcc_mean, mfcc_std, mfcc_max, mfcc_min])

    return feature

    # return mfcc_feature

In [12]:
feature = extractFeature("../myData/00/22307110206-00-03.dat")
print(feature.shape)

(52,)


In [13]:
# 准备数据集
def prepareData(base_dir, classes):
    features = []
    labels = []

    print(f"开始从 {base_dir} 加载数据...")
    
    total_files = 0         # 总文件数
    error_files = 0         # 出错的文件数
    
    # 先计算总文件数
    for idx, name in enumerate(classes):
        class_dir = os.path.join(base_dir, str(idx).zfill(2))
        if os.path.exists(class_dir):
            for filename in os.listdir(class_dir):
                if filename.endswith('.dat'):
                    total_files += 1
    print(f"找到 {total_files} 个文件")
    
    with tqdm(total=total_files, desc="总进度") as pbar:
        for idx, name in enumerate(classes):
            class_dir = os.path.join(base_dir, str(idx).zfill(2))
            
            if not os.path.exists(class_dir):
                print(f"警告：目录 {class_dir} 不存在，跳过类别 '{name}'")
                continue
                
            class_files = [f for f in os.listdir(class_dir) if f.endswith('.dat')]
            print(f"正在处理类别 '{name}' ({len(class_files)}个文件)...")
            
            for filename in class_files:
                filepath = os.path.join(class_dir, filename)
                feature = extractFeature(filepath)
                if feature is not None:
                    features.append(feature)
                    labels.append(idx)
                else:
                    error_files += 1

                pbar.set_postfix({"出错文件数": error_files})
                pbar.update(1)
            
    print(f"\n处理完成: 共 {total_files} 个文件, 出错 {error_files} 个")
    if len(features) == 0:
        raise ValueError("没有成功处理任何文件!请检查数据路径和文件格式")
    
    return np.array(features), np.array(labels)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# 训练SVM模型
def train_svm_model(features, labels, test_size=0.2, random_state=42):
    # 分割训练集和测试集
    x_train, x_test, y_train, y_test = train_test_split(
        features, labels, test_size=test_size, random_state=random_state, stratify=labels
    )
    
    # 特征标准化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(x_train)
    X_test_scaled = scaler.transform(x_test)
    
    # 训练SVM模型
    svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
    svm.fit(X_train_scaled, y_train)
    
    # 评估模型
    y_pred = svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(report)
    
    # 保存模型和标准化器
    joblib.dump(svm, 'svm_model.pkl')
    joblib.dump(scaler, 'scaler.pkl')
    
    return svm, scaler, (X_test_scaled, y_test)

In [15]:
# 使用模型进行识别
def recognize_speech(audio_file, model, scaler, classes):
    # 提取特征
    feature = extractFeature(audio_file)
    if feature is None:
        return "无法检测到语音"
    
    # 标准化特征
    features_scaled = scaler.transform(feature.reshape(1, -1))
    
    # 预测
    probabilities = model.predict_proba(features_scaled)[0]
    class_idx = np.argmax(probabilities)
    confidence = probabilities[class_idx]
    
    return classes[class_idx], confidence

In [16]:
classes = ["数字", "语音", "语言", "处理", "中国", "忠告", "北京", "背景", "上海", "商行",
              "Speech", "Speaker", "Signal", "Sequence", "Processing", "Print", "Project", "File", "Open", "Close"]

# 准备训练数据
# features, labels = prepareData('../myData', classes)
features, labels = prepareData('../myData', classes)

开始从 ../myData 加载数据...
找到 2000 个文件


总进度:   0%|          | 2/2000 [00:00<01:44, 19.07it/s, 出错文件数=0]

正在处理类别 '数字' (100个文件)...


总进度:   5%|▌         | 101/2000 [00:09<01:51, 17.03it/s, 出错文件数=0]

正在处理类别 '语音' (100个文件)...


...

警告: ../myData/19/23300240026-19-07.dat 语音段太短(480采样点)，填充至512采样点


总进度:  98%|█████████▊| 1957/2000 [02:49<00:02, 15.05it/s, 出错文件数=2]

警告: ../myData/19/23300240026-19-13.dat 没有检测到语音段


总进度: 100%|██████████| 2000/2000 [02:52<00:00, 11.57it/s, 出错文件数=2]


处理完成: 共 2000 个文件, 出错 2 个


In [17]:
# 训练模型
svm_model, scaler, test_data = train_svm_model(features, labels)

Accuracy: 0.8650
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.85      0.92        20
           1       0.77      1.00      0.87        20
           2       1.00      0.85      0.92        20
           3       0.79      0.95      0.86        20
           4       1.00      0.90      0.95        20
           5       0.94      0.85      0.89        20
           6       0.69      0.90      0.78        20
           7       0.93      0.70      0.80        20
           8       0.95      0.95      0.95        20
           9       0.94      0.85      0.89        20
          10       0.78      0.90      0.84        20
          11       0.65      0.75      0.70        20
          12       0.90      0.90      0.90        20
          13       1.00      0.85      0.92        20
          14       0.89      0.85      0.87        20
          15       0.82      0.90      0.86        20
          16       0.77      0.85      0.

In [26]:
# 测试识别
test_file = "../records/22300240008-曹奕伦-录音/22300240008-曹奕伦-录音/22300240008-12-09.dat"
predicted_class, confidence = recognize_speech(test_file, svm_model, scaler, classes)
print(f"识别结果: {predicted_class}，置信度: {confidence:.4f}")

识别结果: Signal，置信度: 0.6439
